# Telco Customer Churn for ICP4D

We'll use this notebook to create a machine learning model to predict customer churn.

# 1.0 Install required packages

In [1]:
!pip freeze

absl-py==0.7.1
alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.7
anaconda-project==0.8.2
asn1crypto==0.24.0
astor==0.8.0
astroid==2.2.5
astropy==3.1.2
astunparse==1.6.2
atomicwrites==1.3.0
attrs==19.1.0
autovizwidget==0.12.9
Babel==2.7.0
backcall==0.1.0
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.7.1
bitarray==0.9.3
bkcharts==0.2
blaze==0.11.3
bleach==3.1.0
blinker==1.4
bokeh==1.2.0
boto==2.49.0
boto3==1.9.163
botocore==1.12.163
Bottleneck==1.2.1
brunel==2.6.2
bz2file==0.98
cdsax-jupyter-extensions==0.1
certifi==2019.3.9
cffi==1.12.3
chardet==3.0.4
Click==7.0
clickclick==1.2.2
cloudpickle==1.1.1
clyent==1.2.2
cognitive-assistant==1.0.30
colorama==0.4.1
colorlover==0.3.0
colour==0.1.5
conda==4.6.14
conda-build==3.18.2
conda-package-handling==0+unknown
conda-verify==3.1.1
connexion==2.2.0
contextlib2==0.5.5
coverage==4.5.3
cryptography==2.7
cufflinks==0.15
cx-Oracle==7.1.3
cycler==0.10.0
Cython==0.29.10
cytoolz==0.9.0.1
dask==1.2.2
dat

In [2]:
!pip install --user watson-machine-learning-client --upgrade | tail -n 1

# 2.0 Load and Clean data
We'll load our data as a pandas data frame.

* Highlight the cell below by clicking it.
* Click the `10/01` "Find data" icon in the upper right of the notebook.
* To load the virtualized data created in Exercise-1, choose the `Remote` tab.
* Choose your virtualized data (i.e. User<xyz>.billingProductCustomers), click `Insert to code` and choose `Insert Pandas DataFrame`
* The code to bring the data into the notebook environment and create a Pandas DataFrame will be added to the cell below.
* Run the cell


In [4]:
# Place cursor below and insert the Pandas DataFrame for the Telco churn data

import dsx_core_utils, requests, jaydebeapi, os, io, sys
from pyspark.sql import SparkSession
import pandas as pd
df2 = None
dataSet = dsx_core_utils.get_remote_data_set_info('USER999.billing+products+customer')
dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])
if (sys.version_info >= (3, 0)):
  conn = jaydebeapi.connect(dataSource['driver_class'], dataSource['URL'], [dataSource['user'], dataSource['password']])
else:
  conn = jaydebeapi.connect(dataSource['driver_class'], [dataSource['URL'], dataSource['user'], dataSource['password']])
query = 'select * from "' + (dataSet['schema'] + '"."' if (len(dataSet['schema'].strip()) != 0) else '') +  dataSet['table'] + '"'

if (dataSet['query']):
    query = dataSet['query']
df2 = pd.read_sql(query, con=conn)
df2.head()




,Churn,Contract,DeviceProtection,InternetService,MonthlyCharges,MultipleLines,OnlineBackup,OnlineSecurity,PaperlessBilling,PaymentMethod,...,StreamingMovies,StreamingTV,TechSupport,TotalCharges,customerID,Dependents,Partner,SeniorCitizen,gender,tenure
0,No,One year,No internet service,No,19.30,No,No internet service,No internet service,No,Credit card (automatic),...,No internet service,No internet service,No internet service,486.20,9347-AERRL,No,Yes,0,Male,23
1,Yes,Month-to-month,Yes,Fiber optic,104.50,Yes,Yes,No,Yes,Credit card (automatic),...,Yes,Yes,No,4036.85,0093-XWZFY,No,No,0,Male,40
2,No,Two year,Yes,DSL,63.10,No phone service,Yes,Yes,Yes,Bank transfer (automatic),...,Yes,Yes,Yes,4685.55,2274-XUATA,No,Yes,1,Male,72
3,Yes,Month-to-month,No,Fiber optic,75.05,Yes,No,No,Yes,Credit card (automatic),...,No,No,No,256.25,1980-KXVPM,No,No,1,Female,3
4,Yes,Month-to-month,Yes,Fiber optic,81.00,Yes,No,No,Yes,Electronic check,...,No,No,No,1917.10,7703-ZEKEF,No,No,0,Male,23


We'll use the Pandas naming convention `df` for our DataFrame.  Make sure that the cell below uses the name for the dataframe used above, i.e df1, df2,... dfX.

In [5]:
df = df2

### 2.1 Drop CustomerID feature (column)

In [6]:
df = df.drop('customerID', axis=1)
df.head(5)

,Churn,Contract,DeviceProtection,InternetService,MonthlyCharges,MultipleLines,OnlineBackup,OnlineSecurity,PaperlessBilling,PaymentMethod,PhoneService,StreamingMovies,StreamingTV,TechSupport,TotalCharges,Dependents,Partner,SeniorCitizen,gender,tenure
0,No,One year,No internet service,No,19.30,No,No internet service,No internet service,No,Credit card (automatic),Yes,No internet service,No internet service,No internet service,486.20,No,Yes,0,Male,23
1,Yes,Month-to-month,Yes,Fiber optic,104.50,Yes,Yes,No,Yes,Credit card (automatic),Yes,Yes,Yes,No,4036.85,No,No,0,Male,40
2,No,Two year,Yes,DSL,63.10,No phone service,Yes,Yes,Yes,Bank transfer (automatic),No,Yes,Yes,Yes,4685.55,No,Yes,1,Male,72
3,Yes,Month-to-month,No,Fiber optic,75.05,Yes,No,No,Yes,Credit card (automatic),Yes,No,No,No,256.25,No,No,1,Female,3
4,Yes,Month-to-month,Yes,Fiber optic,81.00,Yes,No,No,Yes,Electronic check,Yes,No,No,No,1917.10,No,No,0,Male,23


### 2.2 Examine the data types of the features

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
Churn               7043 non-null object
Contract            7043 non-null object
DeviceProtection    7043 non-null object
InternetService     7043 non-null object
MonthlyCharges      7043 non-null float64
MultipleLines       7043 non-null object
OnlineBackup        7043 non-null object
OnlineSecurity      7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
PhoneService        7043 non-null object
StreamingMovies     7043 non-null object
StreamingTV         7043 non-null object
TechSupport         7043 non-null object
TotalCharges        7032 non-null float64
Dependents          7043 non-null object
Partner             7043 non-null object
SeniorCitizen       7043 non-null int64
gender              7043 non-null object
tenure              7043 non-null int64
dtypes: float64(2), int64(2), object(16)
memory usage: 1.1+ MB


### 2.3 Any NaN values should be removed to create a more accurate model. Prior examination shows NaN values for `TotalCharges`

In [8]:
# Check if we have any NaN values
df.isnull().values.any()

True

Set `nan_column` to the column number for TotalCharges (starting at 0).

In [9]:
nan_column = df.columns.get_loc("TotalCharges")
print(nan_column)

14


In [10]:
# Handle missing values for nan_column (TotalCharges)

from sklearn.preprocessing import Imputer

imp = Imputer(missing_values="NaN", strategy="mean")

df.iloc[:, nan_column] = imp.fit_transform(df.iloc[:, nan_column].values.reshape(-1, 1))
df.iloc[:, nan_column] = pd.Series(df.iloc[:, nan_column])

In [11]:
# Check if we have any NaN values
df.isnull().values.any()

False

# 3.0 Create a model

In [12]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.createDataFrame(df)
df_data.head()

Row(Churn='No', Contract='One year', DeviceProtection='No internet service', InternetService='No', MonthlyCharges=19.3, MultipleLines='No', OnlineBackup='No internet service', OnlineSecurity='No internet service', PaperlessBilling='No', PaymentMethod='Credit card (automatic)', PhoneService='Yes', StreamingMovies='No internet service', StreamingTV='No internet service', TechSupport='No internet service', TotalCharges=486.2, Dependents='No', Partner='Yes', SeniorCitizen=0, gender='Male', tenure=23)

### 3.1 Split the data into training and test sets

In [13]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

Number of records for training: 5631
Number of records for evaluation: 1412


### 3.2 Examine the Spark DataFrame Schema
Look at the data types to determine requirements for feature engineering

In [14]:
spark_df.printSchema()

root
 |-- Churn: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- SeniorCitizen: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- tenure: long (nullable = true)



### 3.3 Use StringIndexer to encodes a string column of labels to a column of label indices

In [15]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model


si_gender = StringIndexer(inputCol = 'gender', outputCol = 'gender_IX')
si_Partner = StringIndexer(inputCol = 'Partner', outputCol = 'Partner_IX')
si_Dependents = StringIndexer(inputCol = 'Dependents', outputCol = 'Dependents_IX')
si_PhoneService = StringIndexer(inputCol = 'PhoneService', outputCol = 'PhoneService_IX')
si_MultipleLines = StringIndexer(inputCol = 'MultipleLines', outputCol = 'MultipleLines_IX')
si_InternetService = StringIndexer(inputCol = 'InternetService', outputCol = 'InternetService_IX')
si_OnlineSecurity = StringIndexer(inputCol = 'OnlineSecurity', outputCol = 'OnlineSecurity_IX')
si_OnlineBackup = StringIndexer(inputCol = 'OnlineBackup', outputCol = 'OnlineBackup_IX')
si_DeviceProtection = StringIndexer(inputCol = 'DeviceProtection', outputCol = 'DeviceProtection_IX')
si_TechSupport = StringIndexer(inputCol = 'TechSupport', outputCol = 'TechSupport_IX')
si_StreamingTV = StringIndexer(inputCol = 'StreamingTV', outputCol = 'StreamingTV_IX')
si_StreamingMovies = StringIndexer(inputCol = 'StreamingMovies', outputCol = 'StreamingMovies_IX')
si_Contract = StringIndexer(inputCol = 'Contract', outputCol = 'Contract_IX')
si_PaperlessBilling = StringIndexer(inputCol = 'PaperlessBilling', outputCol = 'PaperlessBilling_IX')
si_PaymentMethod = StringIndexer(inputCol = 'PaymentMethod', outputCol = 'PaymentMethod_IX')


In [16]:
si_Label = StringIndexer(inputCol="Churn", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

### 3.4 Create a single vector

In [17]:
va_features = VectorAssembler(inputCols=['gender_IX',  'SeniorCitizen', 'Partner_IX', 'Dependents_IX', 'PhoneService_IX', 'MultipleLines_IX', 'InternetService_IX', \
                                         'OnlineSecurity_IX', 'OnlineBackup_IX', 'DeviceProtection_IX', 'TechSupport_IX', 'StreamingTV_IX', 'StreamingMovies_IX', \
                                         'Contract_IX', 'PaperlessBilling_IX', 'PaymentMethod_IX', 'TotalCharges', 'MonthlyCharges'], outputCol="features")

### 3.5 Create a pipeline, and fit a model using RandomForestClassifier 
Assemble all the stages into a pipeline. We don't expect a clean linear regression, so we'll use RandomForestClassifier to find the best decision tree for the data.

In [18]:
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[si_gender, si_Partner, si_Dependents, si_PhoneService, si_MultipleLines, si_InternetService, si_OnlineSecurity, si_OnlineBackup, si_DeviceProtection, \
                            si_TechSupport, si_StreamingTV, si_StreamingMovies, si_Contract, si_PaperlessBilling, si_PaymentMethod, si_Label, va_features, \
                            classifier, label_converter])

model = pipeline.fit(train_data)

In [19]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

areaUnderROC = 0.680748


# 4.0 Save test data

Add a unique name for MODEL_NAME and DEPLOYMENT_NAME

In [20]:
MODEL_NAME = "my_name telco churn model"
DEPLOYMENT_NAME = "my_name deployment"

### 4.1 Write the test data without label to a .csv so that we can later use it for batch scoring

In [21]:
write_score_CSV=test_data.toPandas().drop(['Churn'], axis=1)
write_score_CSV.to_csv('../datasets/TelcoCustomerSparkMLBatchScore.csv', sep=',', index=False)

### 4.2 Write the test data to a .csv so that we can later use it for evaluation

In [22]:
write_eval_CSV=test_data.toPandas()
write_eval_CSV.to_csv('../datasets/TelcoCustomerSparkMLEval.csv', sep=',', index=False)

# 5.0 Deploy the model to Watson Machine Learning

### 5.1 Set up Watson Machine Learning client to communicate with Cloud Pak for Data

In [23]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [25]:
wml_credentials = {
  "url": "https://w.x.y.z",
  "instance_id": "icp",
  "username": "uname",
  "password": "password"
}

client = WatsonMachineLearningAPIClient(wml_credentials)
print(client)

## 5.2 List existing deployments and models

In [26]:
client.repository.list_models()
client.deployments.list()

------------------------------------  ------------------------  ------------------------  ---------
GUID                                  NAME                      CREATED                   FRAMEWORK
dc4ebfa4-5b58-4567-8936-2291e0b022e0  telco churn model         2019-09-16T15:08:57.382Z  mllib-2.3
82eaa056-5cb2-421b-866f-1b15f1d5a6db  GermanCreditRiskModelICP  2019-09-12T15:54:52.494Z  mllib-2.3
------------------------------------  ------------------------  ------------------------  ---------
------------------------------------  ----------------------------  ------  --------------  ------------------------  ---------  -------------
GUID                                  NAME                          TYPE    STATE           CREATED                   FRAMEWORK  ARTIFACT TYPE
f8e6e947-4088-48fa-82d8-dd6c9832b39b  telco churn model deployment  online  DEPLOY_SUCCESS  2019-09-16T15:10:26.736Z  mllib-2.3  model
4f74117f-d8be-4df6-b874-b57f5a5ea8e0  GermanCreditRiskModelICP      online  DEP

### 5.3 Save model to Watson Machine Learning client

In [27]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM",
               client.repository.ModelMetaNames.NAME: MODEL_NAME}
stored_model = client.repository.store_model(model=model, pipeline=pipeline, meta_props=model_props, training_data=train_data)

### 5.4 Deploy the model to Watson Machine Learning

In [28]:
model_uid = client.repository.get_model_uid(stored_model)
print(model_uid)

16d70431-920e-4f8e-b042-eece5aeab428


In [29]:
created_deployment = client.deployments.create(model_uid, name=MODEL_NAME + " deployment")
client.repository.list_models()
client.deployments.list()



#######################################################################################

Synchronous deployment creation for uid: '16d70431-920e-4f8e-b042-eece5aeab428' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='99c87329-eb48-4a61-b50b-c174280d8c84'
------------------------------------------------------------------------------------------------


------------------------------------  -------------------------  ------------------------  ---------
GUID                                  NAME                       CREATED                   FRAMEWORK
16d70431-920e-4f8e-b042-eece5aeab428  scottda telco churn model  2019-09-16T20:13:36.496Z  mllib-2.3
dc4ebfa4-5b58-4567-8936-2291e0b022e0  telco churn model          2019-09-16T15:08:57.

## Congratulations, you have created a model based on customer churn data, and deployed it to Watson Machine Learning!